# Procesamiento del Lenguaje Natural (NLP) usando RNNs

Gran parte del código que aparece a continuación procede del [repositorio de A. Géron](http://github/ageron/handson-ml2/). Muchas gracias a él.

# Celdas preparatorias

In [1]:
# Se requiere Python ≥3.5
import sys
assert sys.version_info >= (3, 5)

# El notebook se está ejecutando en Colab o en Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# if IS_COLAB:
#     %pip install -q -U tensorflow-addons
#     %pip install -q -U transformers

# Se requiere Scikit-Learn ≥0.20
import sklearn
assert sklearn.__version__ >= "0.20"

# Se requiere TensorFlow ≥2.0
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
tf.keras.backend.clear_session()

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Importaciones comunes
import numpy as np
import os

# Para hacer que las salidas sean estable en distintas ejecuciones
np.random.seed(42)
tf.random.set_seed(42)

# Plotear imágenes lindas
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Donde salvar las figuras
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# De a un caracter

La  tarea de PLN que emprenderemos es la predicción/producción de texto de un carácter a la vez. En otras palabras, proporcionaremos al modelo un texto como "Hola, mund" y esperaremos que la predicción sea un solo carácter, "o".

Como veremos, la principal dificultad es la preparación de los datos; el resto son RNN a la antigua usanza. Vamos a sumergirnos en la clase `Dataset` de `Tensorflow`.

## Dataset

Hay varias maneras de crear un objeto `Dataset`, pero quizás la más fácil es usar `from_tensor_slices`.

In [2]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
X

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

De forma equivalente:

In [4]:
dataset = tf.data.Dataset.range(10)
dataset

<_RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

En cualquier caso, la instancia resultante puede utilizarse como un iterador

In [5]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


#### Repeat y Batch

Esta clase tiene muchos métodos interesantes, como `repeat` o `batch`.

In [6]:
dataset = dataset.repeat(3).batch(7, drop_remainder=True)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)


### Map

In [7]:
dataset = dataset.map(lambda x: x * 2)

In [8]:
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int64)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int64)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int64)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int64)


### Unbatch

In [9]:
#dataset = dataset.apply(tf.data.experimental.unbatch()) # Now deprecated
dataset = dataset.unbatch()

**Pregunta**. ¿Qué va a devolver el siguiente código?

In [ ]:
for item in dataset:
    print(item)

### Filter

In [ ]:
dataset = dataset.filter(lambda x: x < 10)  # keep only items < 10

**Pregunta**. ¿Y ahora?

In [ ]:
for item in dataset:
    print(item)

### Take

In [ ]:
for item in dataset.take(3):
    print(item)

### Shuffle

Miren cómo funciona `buffer_size`, que es bastante sutil; se toma un conjunto de buffer_size elementos, y se elije uno al azar; luego se tomar un conjunto de los siguientes buffer_size elementos, incluyendo los que no quedaron seleccionados en el primer caso, y se vuelve a elegir.

Prueben distintos valores de este parámetro para entenderlo.

In [10]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=3000, seed=42).batch(10)
for item in dataset:
    print(item)

tf.Tensor([5 4 7 3 6 5 5 2 2 7], shape=(10,), dtype=int64)
tf.Tensor([6 8 8 1 1 4 9 9 0 9], shape=(10,), dtype=int64)
tf.Tensor([3 0 2 8 0 1 3 7 4 6], shape=(10,), dtype=int64)


### Window

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))

window_length = 3
dataset =  dataset.window(window_length, shift=4, drop_remainder=False)
for item in dataset:
    print(item)
    for ii in item:
        print(ii)

Fíjense la estructura *anidada* (*nested*) de este dataset.

### Flat map

Este método es especialmente útil para conjuntos de datos anidados, como los anteriores. Transforma un conjunto de datos anidado en uno plano, pero aplicando antes una función a cada conjunto de datos anidado.

In [ ]:
# Pasar la función identidad
for item in dataset.flat_map(lambda x: x):
    print(item)

In [ ]:
# Pasar la función de batch con la longitud de la ventana nos da algo que usaremos más tarde.
for item in dataset.flat_map(lambda x: x.batch(window_length)):
    print(item)

***

Listos para avanzar.

## Dividir una secuencia en lotes de ventanas ordenadas al azar

Por ejemplo, dividamos la secuencia 0 a 14 en ventanas de longitud 5, cada una desplazada por 2 (por ejemplo, `[0, 1, 2, 3, 4]`, `[2, 3, 4, 5, 6]`, etc.), luego barajémoslas, y dividámoslas en entradas (los primeros 4 pasos) y objetivos (los últimos 4 pasos) (por ejemplo `[2, 3, 4, 5, 6]` se dividiría en `[[2, 3, 4, 5], [3, 4, 5, 6]]`), y luego se crearían lotes de 3 de estos pares de entrada/objetivo:

In [11]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
# dataset = tf.data.Dataset.range(15)

In [12]:
dataset = dataset.window(n_steps, shift=2, drop_remainder=False)

# for item in dataset:
#     print(item)
#     for ii in item:
#         print(ii)


In [13]:
dataset = dataset.flat_map(lambda window: window.batch(n_steps, drop_remainder=True))

for item in dataset:
    print(item)


tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int32)
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int32)
tf.Tensor([ 6  7  8  9 10], shape=(5,), dtype=int32)
tf.Tensor([ 8  9 10 11 12], shape=(5,), dtype=int32)
tf.Tensor([10 11 12 13 14], shape=(5,), dtype=int32)


In [14]:
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))

for item in dataset:
    print(item)

(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([6, 7, 8, 9], dtype=int32)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 7,  8,  9, 10], dtype=int32)>)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([2, 3, 4, 5], dtype=int32)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([3, 4, 5, 6], dtype=int32)>)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 5, 6, 7], dtype=int32)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([5, 6, 7, 8], dtype=int32)>)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 2, 3, 4], dtype=int32)>)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 8,  9, 10, 11], dtype=int32)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 9, 10, 11, 12], dtype=int32)>)
(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([10, 11, 12, 13], dtype=int32)>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([11, 12, 13, 14], dtype=int32)>)


In [15]:
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 5, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

_____ Batch 0 
X_batch
[[ 4  5  6  7]
 [ 0  1  2  3]
 [ 8  9 10 11]]
===== 
Y_batch
[[ 5  6  7  8]
 [ 1  2  3  4]
 [ 9 10 11 12]]
_____ Batch 1 
X_batch
[[10 11 12 13]
 [ 2  3  4  5]
 [ 6  7  8  9]]
===== 
Y_batch
[[11 12 13 14]
 [ 3  4  5  6]
 [ 7  8  9 10]]


In [16]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 5, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

_____ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
_____ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


## Cargamos los datos y preparamos el Dataset

Trabajaremos con las obras de Shakespeare del famoso [blog de Karpathy](https://karpathy.github.io/2015/05/21/rnn-effectiveness/).

In [17]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [18]:
print(shakespeare_text[150000:150000+148])

ords.

All The Lords:
You are most welcome home.

AUFIDIUS:
I have not deserved it.
But, worthy lords, have you with heed perused
What I have writte


Veamos qué caracteres aparecen en el texto.

In [19]:
len(shakespeare_text)

1115394

In [20]:
" ".join(sorted(set(shakespeare_text.lower())))

"\n   ! $ & ' , - . 3 : ; ? a b c d e f g h i j k l m n o p q r s t u v w x y z"

Primero debemos convertir estos caracteres en números. Esto es muy fácil de hacer con la clase `Tokenizer` que también es útil para muchas otras tareas de PNL.

Aquí lo utilizamos con el argumento `char_level` establecido en `True`, de modo que cada carácter del texto recibe un token, y lo ajustamos utilizando todo el conjunto de datos.

In [21]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(shakespeare_text)

Observe los dos métodos para ir y venir entre los tokens y los caracteres reales. Observen también que hemos descuidado completamente la diferencia entre las letras minúsculas y las mayúsculas. El uso de `lower=False` en el tokenizador revierte esto.

In [22]:
tokenizer.texts_to_sequences(["First"])

[[50, 10, 8, 7, 3]]

In [23]:
# tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])
tokenizer.sequences_to_texts([[50, 10, 8, 7, 3]])

['F i r s t']

In [24]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

print('Hay {} caracteres diferentes en el texto'.format(max_id))
print('Hay un total de {}  caracteres en el texto'.format(dataset_size))

Hay 65 caracteres diferentes en el texto
Hay un total de 1115394  caracteres en el texto


Ahora vamos a tokenizar el arte shakesperiano y convertirlo en un elemento para *nuestro* arte (ahre)

In [25]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

Ahora viene la parte complicada. Es imposible entrenar una RNN con una sola instancia de más de 1 millón de caracteres. En su lugar, dividiremos el texto en pequeñas frases de unos 100 caracteres y las utilizaremos para entrenar la RNN.

Esta es la idea:

![Imagen](../images/Figure16-1_geron.png)

N.B.: ya puedes adivinar cómo se utilizará el método `window`.


In [26]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

Ok, eso fue fácil... pero la RNN no podrá entrenar con un *conjunto de datos anidados*, así que necesitamos usar `flat_map` como antes.

In [27]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Ahora shuffle, batchear y construir las "características" y la etiqueta (es decir, los 100 primeros caracteres y el carácter siguiente correspondiente)

In [28]:
# np.random.seed(42)
# tf.random.set_seed(42)
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)

In [29]:
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Ponemos todo esto en una práctica función.

In [30]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

Además, seguimos tratando con *tokens* (números), que son una variable categórica. Normalmente es necesario codificarlos. Aquí utilizamos la codificación de un solo punto (no hay tantos caracteres distintos). Introduzca el método `map`.

In [31]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

Veamos lo que tenemos. Agarramos el primer lote y miramos las formas.

In [32]:
dataset = dataset.prefetch(1)

for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 65) (32, 100)


Así pues, tenemos 32 instancias, de cien caracteres, cada una de las cuales es un vector de 32 elementos (sólo uno de los cuales no es cero).

Los *targets* no están codificados.

## Creación y entrenamiento del modelo

**Atención**: el siguiente código puede tardar hasta 24 horas en ejecutarse, dependiendo de tu hardware. Si usas una GPU, puede tardar solo 1 o 2 horas, o menos.

**Nota de Géron**: la clase `GRU` sólo utilizará la GPU (si tiene una) cuando utilice los valores por defecto para los siguientes argumentos: `activation`, `recurrent_activation`, `recurrent_dropout`, `unroll`, `use_bias` y `reset_after`. Por eso he comentado `recurrent_dropout=0.2` (en comparación con el libro).

In [33]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(dataset, epochs=10)

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         74880     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed (TimeDist  (None, None, 65)          8385      
 ributed)                                                        
                                                                 
Total params: 182337 (712.25 KB)
Trainable params: 182337 (712.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
model = keras.models.load_model('models/NLP_char.h5')

OSError: ignored

## Usando el modelo para generar texto

Construimos una pequeña función para preprocesar cualquier texto que pasemos al modelo.

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

**Atención**: el método `predict_classes()` está obsoleto. En su lugar, debemos utilizar `np.argmax(model(X_new), axis=-1)`.

In [ ]:
X_new = preprocess(["The King is dea"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)

# Suma 1 porque los tokens van de 1 en adelante (el cero es para enmascarar)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

In [ ]:
(np.argmax(model.predict(X_new), axis=-1) + 1)[0][-1]

¡Éxito! Juguemos con otros textos.

In [ ]:
X_new = preprocess(["It's only machine learning but I like i"])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

Produzcamos ahora una secuencia de caracteres, uno tras otro, para hacer nuevos textos completos.

In [ ]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
tf.random.set_seed(42)

next_char("How are yo", temperature=20)

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)

print(complete_text("t", n_chars=100, temperature=0.2))

In [ ]:
print(complete_text("t", temperature=1))

In [ ]:
print(complete_text("t", temperature=2))

In [ ]:
print(complete_text(shakespeare_text[:100], n_chars=100, temperature=0.2))

In [ ]:
print(shakespeare_text[:100])

## * Stateful RNN

In [ ]:
tf.random.set_seed(42)

Las RNN con estado (Stateful RNN) mantienen los valores de estado de una instancia a la siguiente, lo que permite aprender secuencias mucho más largas.

El principal problema es la preparación del conjunto de datos, especialmente si queremos agrupar las instancias por lotes. Para evitar este problema, ahora utilizamos una única instancia por lote.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

# Cambiar esto requiere mucho trabajo!
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

Al final de cada época, queremos volver a los estados vacíos y empezar de nuevo. Esto se hace con un `Callback` casero.

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Para utilizar el modelo con diferentes tamaños de lote, necesitamos crear una copia sin estado. Podemos deshacernos del abandono, ya que sólo se utiliza durante el entrenamiento:

In [ ]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

Para fijar los pesos, primero tenemos que construir el modelo (para que se creen los pesos):

In [ ]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [ ]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [ ]:
tf.random.set_seed(42)

print(complete_text("t"))